<a href="https://colab.research.google.com/github/abialbon/pytorch-udacity-scholarship/blob/master/Project/Species_classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from google.colab import drive
drive.mount('/content/drive')

In [0]:
def logger(text, log_path):
    with open(log_path, 'a') as f:
        f.write(text + '\n')
        
log_path = 'drive/My Drive/job_/log.txt'

In [0]:
# http://pytorch.org/
from os.path import exists
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())
cuda_output = !ldconfig -p|grep cudart.so|sed -e 's/.*\.\([0-9]*\)\.\([0-9]*\)$/cu\1\2/'
accelerator = cuda_output[0] if exists('/dev/nvidia0') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision



In [0]:
# All imports
import time
import torch
import numpy as np
from torchvision import datasets, models, transforms
from torch import nn
import torch.nn.functional as F
from torch import optim

device = 'cuda' if torch.cuda.is_available() else 'cpu'
device
print(torch.__version__)

0.4.1


In [0]:
!wget -cq https://s3.amazonaws.com/content.udacity-data.com/courses/nd188/flower_data.zip

In [0]:
!wget -cq https://raw.githubusercontent.com/udacity/pytorch_challenge/master/cat_to_name.json

In [0]:
!unzip -qq flower_data.zip

In [0]:
import json

with open('cat_to_name.json', 'r') as f:
    classes_to_labels = json.load(f)

In [0]:
image_dir = 'flower_data'
train = '/train'
valid = '/valid'

train_transforms = transforms.Compose([transforms.Resize(256),
                                      transforms.CenterCrop(224),
                                      transforms.ToTensor(),
                                      transforms.Normalize(mean = [0.485, 0.456, 0.406], std = [0.229, 0.224, 0.225])])

trainset = datasets.ImageFolder(image_dir + train, transform=train_transforms)
valset = datasets.ImageFolder(image_dir + valid, transform=train_transforms)

train_loader = torch.utils.data.DataLoader(trainset, batch_size=32, shuffle=True)
val_loader = torch.utils.data.DataLoader(valset, batch_size=32, shuffle=True)

In [0]:
%%capture
fc = nn.Sequential(nn.Linear(2048, len(classes_to_labels)),
                  nn.LogSoftmax(dim=1))

model = models.resnet152(pretrained=True)

model.fc = fc
model.to(device)

In [0]:
criteria = nn.NLLLoss()
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9)
scheduler = torch.optim.lr_scheduler.StepLR(optimizer, step_size=5, gamma=0.1)

In [0]:
def save_model(model):
    dict_to_save = {
        'class_to_idx': trainset.class_to_idx,
        'input_layers': 2048,
        'output_layers': len(classes_to_labels),
        'state_dict': model.cpu().state_dict()
    }
    torch.save(dict_to_save, 'drive/My Drive/job_/model_resent_new.pt')

In [0]:
def train(n_epochs, model, criteria, optimizer, log_path):
    model.train()
    for e in range(n_epochs):
        start = time.time()
        train_loss = 0
        for images, labels in train_loader:
            images, labels = images.to(device), labels.to(device)
            
            output = model.forward(images)
            loss = criteria(output, labels)
            train_loss += loss.item()
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
        else:
            val_loss = 0
            accuracy = 0
            model.eval()
            with torch.no_grad():
                for images, labels in val_loader:
                    images, labels = images.to(device), labels.to(device)

                    output = model.forward(images)
                    loss = criteria(output, labels)
                    val_loss += loss.item()
                    probs = torch.exp(output)
                    prob, top_c = probs.topk(1)
                    
                    equals = top_c.squeeze() == labels
                    equals = equals.type(torch.FloatTensor).mean()
                    accuracy += equals.item()
                else:
                    end = time.time()
                    total_time = end - start
                    metric_string = 'Epoch: {:2d}/{:2d} ---- Train loss: {:3.3f} ---- Val loss: {:3.3f} ---- Acc: {:3.2f}%'.format(e+1, n_epochs, train_loss/len(train_loader), val_loss/len(val_loader), (accuracy/len(val_loader)) * 100)
                    try:
                        logger(metric_string, log_path)
                    finally:
                        print(metric_string)
                        print('Time for last epoch: {:2d}m {:2d}s'.format(int(total_time//60), int(total_time % 60)))
                    model.train()
    scheduler.step()

In [0]:
train(15, model, criteria, optimizer, log_path)

Epoch:  1/15 ---- Train loss: 3.174 ---- Val loss: 1.592 ---- Acc: 70.90%
Time for last epoch:  8m 56s
Epoch:  2/15 ---- Train loss: 1.052 ---- Val loss: 0.535 ---- Acc: 92.84%
Time for last epoch:  8m 57s
Epoch:  3/15 ---- Train loss: 0.397 ---- Val loss: 0.267 ---- Acc: 96.39%
Time for last epoch:  8m 57s
Epoch:  4/15 ---- Train loss: 0.200 ---- Val loss: 0.191 ---- Acc: 96.75%
Time for last epoch:  8m 57s
Epoch:  5/15 ---- Train loss: 0.110 ---- Val loss: 0.156 ---- Acc: 97.53%
Time for last epoch:  8m 57s
Epoch:  6/15 ---- Train loss: 0.070 ---- Val loss: 0.129 ---- Acc: 98.08%
Time for last epoch:  8m 57s
Epoch:  7/15 ---- Train loss: 0.050 ---- Val loss: 0.121 ---- Acc: 98.08%
Time for last epoch:  8m 57s
Epoch:  8/15 ---- Train loss: 0.039 ---- Val loss: 0.111 ---- Acc: 98.32%
Time for last epoch:  8m 56s
Epoch:  9/15 ---- Train loss: 0.032 ---- Val loss: 0.103 ---- Acc: 98.46%
Time for last epoch:  8m 56s
Epoch: 10/15 ---- Train loss: 0.025 ---- Val loss: 0.101 ---- Acc: 98.32%

In [0]:
save_model(model)